 ## <center>     <center>Зачетное задание

<font size=3>  *Модуль: "Основы машинного обучения "*
<font size=3>  *Слушатель: Турчин М.В.*<br>
<font size=3> *Поток: 7*<br>

    
    

ЗАДАНИЕ:

1. Обучить группу моделей (**Logistic Regression, Decision Tree, Random Forest, Gradient Boosting, MLPClassifier**)
с поиском их параметров с помощью данных обучающей выборки train_5_students.csv. 
У каждого алгоритма машинного обучения есть свой параметр: например, у Decision Tree это max_depth.
Перебор параметров делать по сетке адекватного размера (например, от 1 до 50), чтобы это не заняло слишком много времени.

2. Отобранные модели отсортировать по качеству их предсказания на тестовой выборке test_5_students.csv. Качество предсказания оценивать с помощью метрики точности: accuracy.
3. Выбрать наилучшую модель из получившихся!<br>
Требования к точности модели<br>
- 80% — нормально<br>
- от 80  до 85 — хорошо<br>
- от 85 до 90 — отлично<br>
- от 90 до 100 — идеально<br>


<a id='section_id_0'></a>
### Содержание:
1. **[Подготовка исходных данных](#section_id_1)**
2. **[Получим базовое решение](#section_id_2)**
3. **[EDA](#section_id_3)**
4. **[Обучение и валидация моделей](#section_id_4)**
5. **[Выводы](#section_id_5)**


### 1. Подготовка исходных данных <a id='section_id_1'></a>

* Загрузим библиотеки
* Создадим датафрейм с исходными данными <br>
 ([назад к  меню](#section_id_0))

In [1]:
import pandas as pd
import numpy as np

import random
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
# from lightgbm.sklearn import LGBMClassifier


from sklearn.metrics import recall_score, accuracy_score, f1_score, precision_score

import warnings
warnings.filterwarnings('ignore')

In [2]:
# загрузим данные обучающей и тестовой выборок:
train_df = pd.read_csv('https://drive.google.com/uc?export=download&id=1um61JjG8YBujtOjd_9WpqP7IK4RX0RFA', sep=',')
test_df = pd.read_csv('https://drive.google.com/uc?export=download&id=11tSIBJpQ5285hS8ehMRO7oJ6SbzNN3G4', sep=',')

In [3]:
print('train_df.shape ', train_df.shape)
print('test_df.shape ', test_df.shape)

train_df.shape  (713, 12)
test_df.shape  (178, 12)


 <hr>

### 2. Получим базовое решение  <a id='section_id_2'></a>

* Обучим модели (Logistic Regression, Decision Tree, Random Forest, Gradient Boosting, MLPClassifier) 
* Посчитаем метрику качества <br>

([назад к  меню](#section_id_0))

В качестве базового решения примем модель для дерева решений (DecisionTreeClassifier), полученную на лекции.
Параметры модели:<br> max_depth = 3, acc_test = 0.825843<br>
Минимальное решение по accuracy получено (82%). Дальше должны его улучшить.

 <hr>

### 3. Предобработка данных  <a id='section_id_3'></a>

* Заполним пропуски 
* Создадим новые признаки <br>

([назад к  меню](#section_id_0))

In [4]:
#Замена пола на численные значения
train_df['Sex'].replace({'male':0, 'female':1}, inplace=True)
test_df['Sex'].replace({'male':0, 'female':1}, inplace=True)

In [5]:
train_EDA = train_df.copy()
test_EDA = test_df.copy()

In [6]:
# Найдем пропуски:
train_EDA.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            144
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          555
Embarked         2
dtype: int64

#### Заполним пропуски по Age:

In [7]:
def transform_Age(X):
    Age_mean_df = pd.DataFrame(train_EDA.groupby(['Sex', 'Pclass']).Age.mean())
    X['_Age'] = 0
    X.loc[X.Age.isna(), '_Age'] = 1
    for Pclass in [1,2,3]:
        for sex in [0,1]:
            X.loc[(X.Age.isna()) & (X.Sex == sex) & (X.Pclass == Pclass), 'Age'] =\
                         Age_mean_df.loc[sex, Pclass].values[0]
    return X

In [8]:
train_EDA = transform_Age(train_EDA)
test_EDA = transform_Age(test_EDA)
# train_EDA.loc[train_EDA._Age == 1].head()

In [9]:
# test_EDA.loc[test_EDA._Age == 1].head()

#### Введем новый признак, связанный с Cabin:

Если данные по каюте есть - признак = 1, если нет = 0

In [10]:
def transform_Cabin(X):
    X['_Cabin_flag'] = 1
    X.loc[X.Cabin.isna(), '_Cabin_flag'] = 0
    return X

In [11]:
train_EDA = transform_Cabin(train_EDA)
test_EDA = transform_Cabin(test_EDA)

In [12]:
# train_EDA.loc[train_EDA._Cabin_flag == 1].head()

#### Введем новые признаки, связанные с Embarked:

In [13]:
# какие значения в Embarked?
train_EDA.Embarked.value_counts()

S    510
C    139
Q     62
Name: Embarked, dtype: int64

In [14]:
# Заменим пропуски на самый частый
def transform_Embarked(X):
    X.Embarked.fillna('S', inplace=True)
    X = pd.concat([X, pd.get_dummies(X['Embarked'])], axis=1)
    return X

In [15]:
train_EDA = transform_Embarked(train_EDA)
test_EDA = transform_Embarked(test_EDA)
# test_EDA.head()

In [16]:
# train_EDA.head()

#### Цена билета

In [17]:
# Введем дополнительные признаки по стоимости билета (разобъем по квантилям)
def fare_to_cat(X):

    X['_fare_cat'] = 0
    X.loc[X['Fare'] >= train_EDA['Fare'].quantile(0.75), '_fare_cat'] = 1
    X.loc[(X['Fare'] < train_EDA['Fare'].quantile(0.75)) & (X['Fare'] > train_EDA['Fare'].quantile(0.5)), '_fare_cat'] = 2
    X.loc[(X['Fare'] < train_EDA['Fare'].quantile(0.5)) & (X['Fare'] > train_EDA['Fare'].quantile(0.25)), '_fare_cat'] = 3
    X.loc[X['Fare'] < train_EDA['Fare'].quantile(0.25), '_fare_cat'] = 4
    return X

In [18]:
train_EDA = fare_to_cat(train_EDA)
test_EDA = fare_to_cat(test_EDA)

#### SibSp и Parch

In [19]:
def transform_SibSp_Parch(X):
    X['_SibSp_Parch'] = 1
    X.loc[(X.SibSp == 0) & (X.Parch == 0), '_SibSp_Parch'] = 0
    return X

In [20]:
train_EDA = transform_SibSp_Parch(train_EDA)
test_EDA = transform_SibSp_Parch(test_EDA)

#### SibSp и Parch (*)

In [21]:
def transform_SibSp_on_Parch(X):
    X['_SibSp_on_Parch'] = 0
    X.loc[(X.SibSp != 0) & (X.Parch != 0), '_SibSp_on_Parch'] = X.SibSp * X.Parch
    return X

In [22]:
train_EDA = transform_SibSp_on_Parch(train_EDA)
test_EDA = transform_SibSp_on_Parch(test_EDA)

In [23]:
train_EDA = transform_SibSp_Parch(train_EDA)
test_EDA = transform_SibSp_Parch(test_EDA)

In [24]:
# train_EDA.loc[(train_EDA._SibSp_on_Parch != 0)]

In [25]:
#выберем признаки для внесения в модель
feature_names = ['Pclass', 
                 'Survived', 
                 'Sex', 
                 'Age', 
                   'SibSp',
                   'Parch',
                 '_SibSp_on_Parch',
                 #'Ticket', 
                 'Fare', 
                 #'Cabin', 
                 #'Embarked', 
                 '_Age', 
                 '_Cabin_flag',
                   'C', 'Q', 'S', 
                 '_fare_cat',
                 '_SibSp_Parch']

In [26]:
train_df = train_EDA[feature_names]
test_df = test_EDA[feature_names]

In [27]:
train_X = train_df.drop(columns=['Survived']) 
train_y = train_df['Survived']

test_X = test_df.drop(columns=['Survived']) 
test_y = test_df['Survived']

### 4. Обучим модели  <a id='section_id_4'></a>

* обучим модели  RandomForestClassifier, GradientBoostingClassifier, DecisionTreeClassifier, LGBMClassifier<br>

([назад к  меню](#section_id_0))

In [31]:
%%time
#LogisticRegression
clf = LogisticRegression(random_state=42)
clf.fit(train_X, train_y)
preds_train = clf.predict(train_X) # предсказания для train_X 
preds_test = clf.predict(test_X) # предсказания для test_X

print('####### РЕЗУЛЬТАТЫ ДЛЯ ОБУЧЕНИЯ #######')
print('Recall: ', recall_score(train_y, preds_train))
print('Accuracy: ', accuracy_score(train_y, preds_train))
print('Precision: ', precision_score(train_y, preds_train))
print('F1 score: ', f1_score(train_y, preds_train))

print('####### РЕЗУЛЬТАТЫ ДЛЯ ТЕСТА #######')
print('Recall: ', recall_score(test_y, preds_test))
print('Accuracy: ', accuracy_score(test_y, preds_test))
print('Precision: ', precision_score(test_y, preds_test))
print('F1 score: ', f1_score(test_y, preds_test))

####### РЕЗУЛЬТАТЫ ДЛЯ ОБУЧЕНИЯ #######
Recall:  0.7328519855595668
Accuracy:  0.8134642356241234
Precision:  0.7748091603053435
F1 score:  0.7532467532467533
####### РЕЗУЛЬТАТЫ ДЛЯ ТЕСТА #######
Recall:  0.7538461538461538
Accuracy:  0.8033707865168539
Precision:  0.7205882352941176
F1 score:  0.7368421052631577
Wall time: 47.4 ms


In [33]:
%%time
#(DecisionTreeClassifier)

results = pd.DataFrame()
for max_depth in range(1, 50, 2):
    clf = DecisionTreeClassifier(random_state=42, max_depth=max_depth)
    clf.fit(train_X, train_y)
    preds_train = clf.predict(train_X)
    preds_test = clf.predict(test_X)
    
    results_dict = {
                    'max_depth':[max_depth],  
                    'recall_train':[recall_score(train_y, preds_train)], 
                    'acc_train':[accuracy_score(train_y, preds_train)],
           
                    'acc_test':[accuracy_score(test_y, preds_test)]         
                   }
    
    results_df = pd.DataFrame.from_dict(results_dict)
    results = results.append(results_df)

results.reset_index(inplace=True, drop=True)

Wall time: 147 ms


In [34]:

results[results.acc_test == results.acc_test.max()]

,max_depth,recall_train,acc_train,acc_test
4,9,0.830325,0.915849,0.853933


In [35]:
%%time
#RandomForestClassifier

results = pd.DataFrame()
for max_depth in range(1, 50, 2):
    clf = RandomForestClassifier(random_state=42, max_depth=max_depth)
    clf.fit(train_X, train_y)
    preds_train = clf.predict(train_X)
    preds_test = clf.predict(test_X)
    
    results_dict = {
                    'max_depth':[max_depth],  
                    'recall_train':[recall_score(train_y, preds_train)], 
                    'acc_train':[accuracy_score(train_y, preds_train)],
           
                    'acc_test':[accuracy_score(test_y, preds_test)]         
                   }
    
    results_df = pd.DataFrame.from_dict(results_dict)
    results = results.append(results_df)

results.reset_index(inplace=True, drop=True)

Wall time: 3.55 s


In [36]:

results[results.acc_test == results.acc_test.max()]

,max_depth,recall_train,acc_train,acc_test
5,11,0.920578,0.962132,0.859551


In [37]:
%%time
# GradientBoostingClassifier

results = pd.DataFrame()
for max_depth in range(1, 50, 2):
    clf = GradientBoostingClassifier(random_state=42, max_depth=max_depth)
    clf.fit(train_X, train_y)
    preds_train = clf.predict(train_X)
    preds_test = clf.predict(test_X)
    
    results_dict = {
                    'max_depth':[max_depth],  
                    'recall_train':[recall_score(train_y, preds_train)], 
                    'acc_train':[accuracy_score(train_y, preds_train)],
           
                    'acc_test':[accuracy_score(test_y, preds_test)]         
                   }
    
    results_df = pd.DataFrame.from_dict(results_dict)
    results = results.append(results_df)

results.reset_index(inplace=True, drop=True)

Wall time: 12.6 s


In [38]:

results[results.acc_test == results.acc_test.max()]

,max_depth,recall_train,acc_train,acc_test
3,7,0.974729,0.987377,0.837079
4,9,0.974729,0.987377,0.837079


In [39]:
%%time
#MLPClassifier
nn = MLPClassifier(random_state=42)
nn.fit(train_X, train_y)
preds_train = nn.predict(train_X) # предсказания для train_X 
preds_test = nn.predict(test_X) # предсказания для test_X


results = pd.DataFrame()
for hidden_size in range(1, 15):
    nn = MLPClassifier(hidden_layer_sizes=(hidden_size, ), random_state=42)
    nn.fit(train_X, train_y)
    preds_train = nn.predict(train_X)
    preds_test = nn.predict(test_X)
    
    results_dict = {
                    'hidden_size':[hidden_size],
                    'recall_train':[recall_score(train_y, preds_train)], 
                    'acc_train':[recall_score(train_y, preds_train)],
                
                    'acc_test':[accuracy_score(test_y, preds_test)]
      
                   }
    
    results_df = pd.DataFrame.from_dict(results_dict)
    results = results.append(results_df)

results.reset_index(inplace=True, drop=True)

Wall time: 3.61 s


In [40]:

results[results.acc_test == results.acc_test.max()]

,hidden_size,recall_train,acc_train,acc_test
6,7,0.606498,0.606498,0.775281
9,10,0.584838,0.584838,0.775281
11,12,0.653430,0.653430,0.775281
13,14,0.653430,0.653430,0.775281


### 5. Выводы  <a id='section_id_5'></a>

* 

([назад к  меню](#section_id_0))

**Вывод:
    из опробованных моделей наилучший результат дает модель DecisionTreeClassifier с результатом 0.853933**

### 6. Дополнение  <a id='section_id_5'></a>

* Немного улучшить результат удалось применив LGBMClassifier

([назад к  меню](#section_id_0))


In [41]:
from lightgbm.sklearn import LGBMClassifier

In [43]:
%%time
# Пример для RandomForestClassifier
dic_model = [RandomForestClassifier, GradientBoostingClassifier, DecisionTreeClassifier, LGBMClassifier]
results = pd.DataFrame()
for max_depth in range(1, 50, 2):
    for class_model in dic_model:
        clf = class_model(random_state=42, max_depth=max_depth)
        clf.fit(train_X, train_y)

        preds_train = clf.predict(train_X)
        preds_test = clf.predict(test_X)
        if class_model == RandomForestClassifier:
            model_name = 'RandomFores'
        elif class_model == GradientBoostingClassifier:
            model_name = 'GradientBoosting'
        elif class_model == DecisionTreeClassifier:
            model_name = 'DecisionTre'   
        else: model_name = 'LGBMClassifier'
        results_dict = {'model': [model_name],
                        'max_depth':[max_depth],  
                        'acc_train':[accuracy_score(train_y, preds_train)],
                        'acc_test':[accuracy_score(test_y, preds_test)]                    
                       }

        results_df = pd.DataFrame.from_dict(results_dict)
        results = results.append(results_df)

results.reset_index(inplace=True, drop=True)

Wall time: 17.1 s


In [29]:
# Получим датафрейм с перечнем моделей и их характеристик(глубина)
results

In [44]:
# выберем лучшую модель по показателю accuracy

results[results.acc_test == results.acc_test.max()]

,model,max_depth,acc_train,acc_test
15,LGBMClassifier,7,0.917251,0.865169


In [28]:
# ************************************